In [1]:
import numpy as np

In [2]:
def movie_dict(arr):
    unique, counts = np.unique(arr, return_counts=True)
    k=len(counts)
    L=len(arr)
    movie_id_dict = {}

    for movie_id in arr:
        movie_id_dict[str(movie_id)] = -1
    for i,key in enumerate(movie_id_dict.keys()):
        movie_id_dict[key] = i
    return movie_id_dict
def get_movies_for_user(user,data,movie_mapping):
    data = data[data[:,0]==user]
    movies_user_seen=[]
    for movie in data[:,1]:
        movies_user_seen.append(movie_mapping[str(movie)])
    return movies_user_seen


def one_hot_encoding(data):
    #data = data.astype(int)
    L = data.shape[0]
    N = int(np.max(data[:,0]))
    unique, counts = np.unique(data[:,1], return_counts=True)
    k=len(counts)
    matrix_encoding_user = np.zeros((L,N))
    matrix_encoding_movie = np.zeros((L,k))
    matrix_encoding_other_movies_rated = np.zeros((L,k))
    matrix_encoding_last_movie_rated = np.zeros((L,k))
    movie_mapping = movie_dict(data[:,1])
    matrix_encoding_time = np.array(data[:,2])

    for i,user in enumerate(data[:,0]):
        matrix_encoding_user[i,int(user)-1] = 1
    for i,movie in enumerate(data[:,1]):
        matrix_encoding_movie[i,movie_mapping[str(movie)]] = 1
    old_user=-1
    for i,user in enumerate(data[:,0]):
        if old_user!=user:
            user_movies = get_movies_for_user(user,data,movie_mapping)
        matrix_encoding_other_movies_rated[i,user_movies] = 1/len(user_movies)
    old_user=-1
    for i,user in enumerate(data[:,0]):
        if old_user==user:
            matrix_encoding_last_movie_rated[i,:] = matrix_encoding_movie[i-1,:]
        old_user=user
    
    
    full_matrix = np.concatenate((matrix_encoding_user,matrix_encoding_movie,matrix_encoding_other_movies_rated,matrix_encoding_time[:, None],matrix_encoding_last_movie_rated),axis=1)


    """
    full_matrix = np.concatenate((matrix_encoding_user,matrix_encoding_movie),axis=1)
    print(full_matrix.shape)
    full_matrix = np.concatenate((full_matrix,matrix_encoding_other_movies_rated),axis=1)
    print(full_matrix.shape)
    print(matrix_encoding_time.shape)
    full_matrix = np.concatenate((full_matrix,matrix_encoding_time[:, None]),axis=1)
    print(full_matrix.shape)
    full_matrix = np.concatenate((full_matrix,matrix_encoding_last_movie_rated),axis=1)
    """
    return full_matrix



In [3]:
filename="ml-latest-small/ratings.csv"
data=np.genfromtxt(filename,skip_header=1,delimiter=",")
unique, counts = np.unique(data[:,1], return_counts=True)
full_matrix_test = one_hot_encoding(data)



In [6]:
full_matrix_test.shape
#print(len(counts))

(100836, 29783)